In [0]:
import bs4 as bs
import urllib.request
import pandas as pd
from time import sleep
from lxml import html  
import requests
from time import sleep
import json
import argparse
from collections import OrderedDict
import lxml.html as lh
import pandas as pd
import urllib3

In [0]:
# Brute force way: not used
def getSummaryData(url):
  response = requests.get(url, verify=False)
  print ("Parsing %s"%(url))
  sleep(4)
  parser = html.fromstring(response.text)
  summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
  summary_data = OrderedDict()
  try:
    for table_data in summary_table:
      raw_table_key = table_data.xpath('.//td[contains(@class,"C($primaryColor)")]//text()')
      raw_table_value = table_data.xpath('.//td[contains(@class,"Ta(end)")]//text()')
      table_key = ''.join(raw_table_key).strip()
      table_value = ''.join(raw_table_value).strip()
      summary_data.update({table_key:table_value})
    return summary_data
  
  except:
    print ("Failed to parse json response")
    return {"error":"Failed to parse json response"}
  
  

In [0]:
# for parsing yahoo finance data

def getFinanceData(url):
  #urllib.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
  response = requests.get(url, verify=False)
  sleep(4)
  parser = html.fromstring(response.text)
  finance_table = parser.xpath('//div[contains(@data-test,"fin-row")]')
  
  keys = []
  values = []
  
  columnLen = 0 
  for table_data in finance_table:
    raw_table_key = table_data.xpath('.//span[contains(@class,"Va(m)")]//text()')
    if len(raw_table_key) > 1:
      continue
      
    raw_table_value = table_data.xpath('.//span[not(contains(@class,"Va(m)"))]//text()')
    
    # raw_table_key has only single element
    keys.append(raw_table_key[0])
    values.append(raw_table_value)
    columnLen = max(columnLen, len(raw_table_value))
  
  columns = []
  data = []
  
  for row in keys:
    for i in range(1, columnLen + 1):
      columns.append(row + "_" + str(i))
      

  
  for value in values:
    temp = []
    if len(value) < columnLen:
      diff = columnLen - len(value)
      while diff > 0:
        temp.append("N/A")
        diff = diff -1
      
    for item in value:
      temp.append(item)
    
    data.extend(temp)
 
  finance_data = pd.DataFrame()
  
  columns_series = pd.Series(columns, index  = None)
  finance_data = pd.DataFrame([data], columns = columns)
  
  return finance_data
      

In [0]:
def parse_url(url, cmp):
  response = requests.get(url)
  soup = bs.BeautifulSoup(response.text, 'lxml')
  
  listdf = []
  
  for table in soup.find_all('table'):
    listdf.append(parse_html_table(table, cmp))
  
  return listdf
    
def parse_html_table(table, cmp):
  n_columns = 0
  n_rows=0
  column_names = []

  # Find number of rows and columns
  # we also find the column titles if we can
  for row in table.find_all('tr'):

      # Determine the number of rows in the table
      td_tags = row.find_all('td')
      if len(td_tags) > 0:
          n_rows+=1
          if n_columns == 0:
              # Set the number of columns for our table
              n_columns = len(td_tags)

      # Handle column names if we find them
      th_tags = row.find_all('th') 
      if len(th_tags) > 0 and len(column_names) == 0:
          for th in th_tags:
              column_names.append(th.get_text().replace(cmp, "cmp"))
  
  # Safeguard on Column Titles
  if len(column_names) > 0 and len(column_names) != n_columns:
      raise Exception("Column titles do not match the number of columns")

  columns = column_names if len(column_names) > 0 else range(0,n_columns)
  df = pd.DataFrame(columns = columns,
                    index= range(0,n_rows))
  row_marker = 0
  for row in table.find_all('tr'):
      column_marker = 0
      columns = row.find_all('td')
      for column in columns:
          df.iat[row_marker,column_marker] = column.get_text()
          column_marker += 1
      if len(columns) > 0:
          row_marker += 1

  # Convert to float if possible
  for col in df:
      try:
          df[col] = df[col].astype(float)
      except ValueError:
          pass

  return df

In [0]:
cmpList = ['TSLA', 'AAPL', 'NFLX', 'GM', 'F', 'FCAU', 'PCAR', 'MRK', 'PFE', 'JNJ', 'BMY', 'QCOM', 'MSFT' , 'ADBE', 'INTC', 'GOOGL', 'CRM', 'ORCL', 'IBM', 'FB', 'TWTR', 'AMZN', 'SCHW', 'ETFC', 'AMTD',]

In [100]:
# Parsing data from yahoo finance (profile, analysis, summary)

towriteCmpList = []

from google.colab import drive
drive.mount('drive')

#https://finance.yahoo.com/quote/%5EIXIC/components?p=%5EIXIC

items = ['', '/profile', '/analysis']
urlPre = 'https://finance.yahoo.com/quote/'
urlSuf = '?p='

columns = None

final_df = pd.DataFrame()

for cmp in cmpList:
  cmp_df_list = []
  print('\nCompany: ', cmp)
  
  for item in items:
    url = urlPre + cmp + item + urlSuf + cmp
    dataframe_list = parse_url(url, cmp)
    
    if item is '':
      name = 'summary'
    else:
      name = item[1:len(item)]
       
    count = 1
    
    item_df = pd.DataFrame()
    
    
    for df in dataframe_list:
    
      # Summary (2), Analysis (6), Profile (1)
      
      
      if item is "":
        df = df.transpose()
        item_df = pd.concat([item_df, df], axis = 1, ignore_index = True)
        
        
      if item == "/analysis": 
        df.set_index(df.columns[0], inplace=True)
        df = df.replace({pd.np.nan: None})
        df = df.unstack().to_frame().sort_index(level=1).T
        df.columns = df.columns.map("_".join)
        item_df = pd.concat([item_df, df], axis = 1)
        
     
      if item == "/profile":     
        column_Names = df.columns
        columns = []
        rows = df.shape[0]

        for j in range(1, rows + 1):
          for i in column_Names:
            columns.append(str(i) + "_" + str(j))

        columns_series = pd.Series(columns, index  = None)
        values_series = pd.Series(df.values.flatten(), index = None)
        values_list = [df.values.flatten()]
        item_df = pd.DataFrame(values_list, columns = columns)
        
      
    
    if item is "":
      new_header = item_df.iloc[0] #grab the first row for the header
      item_df.columns = new_header #set the header row as the df header
      item_df = item_df[1:] #take the data less the header row
      item_df.reset_index(inplace=True, drop=True)
     
      
    #print(name, len(dataframe_list), 'tables ', item_df.shape[1], 'Columns')
    cmp_df_list.append(item_df)
    
#     filename = cmp + '_' + name + '.csv'
#     item_df.to_csv(filename, index = False)
#     !cp $filename drive/My\ Drive/Isenberg/
  
  cmp_df = pd.DataFrame()
  for df in cmp_df_list:
    cmp_df = pd.concat([cmp_df,df], axis = 1)
  
  # Causes problem when company name is "F" and it gets replaced everywhere in the column names. Better to do in parse_url
  # cmp_df.columns = cmp_df.columns.str.replace(cmp, "cmp")
  
  if columns == None:
    columns = cmp_df.columns.tolist()
 
  #cmp_df.sort_index(axis=1, inplace=True) # not required if string replace done at parsing time
  
  if (final_df.shape[0] != 0 and final_df.columns.equals(cmp_df.columns) == False ):
    # except when final_df is empty. This should return always True. Else column mistmatch (ignore this company)
    print('Column mismatch for company: ', cmp, ' Ignoring...')
    continue
    # MSFT: has current year (2020): typo
    # ADBE: has different dates
  else:
    towriteCmpList.append(cmp)
  
  #print('differences: ', cmp_df.columns.difference(final_df.columns).shape, final_df.columns.difference(cmp_df.columns).shape) #should always print 0
  
  final_df = pd.concat([final_df, cmp_df], ignore_index = True)

final_df['company'] = towriteCmpList  
final_df = final_df.set_index('company')
print('Final Data: ',final_df.shape)

filename = 'YahooData.xlsx'
final_df.to_excel(filename)
!cp $filename drive/My\ Drive/Isenberg/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive

Company:  TSLA

Company:  AAPL

Company:  NFLX
Column mismatch for company:  NFLX  Ignoring...

Company:  GM

Company:  F

Company:  FCAU

Company:  PCAR

Company:  MRK

Company:  PFE

Company:  JNJ
Column mismatch for company:  JNJ  Ignoring...

Company:  BMY

Company:  QCOM

Company:  MSFT
Column mismatch for company:  MSFT  Ignoring...

Company:  ADBE
Column mismatch for company:  ADBE  Ignoring...

Company:  INTC

Company:  GOOGL

Company:  CRM
Column mismatch for

In [24]:
# Parsing data from marketwatch (financials)

towriteCmpList = []

from google.colab import drive
drive.mount('drive')

#https://finance.yahoo.com/quote/%5EIXIC/components?p=%5EIXIC

items = ['/financials']
urlPre = 'https://www.marketwatch.com/investing/stock/'

columns = None


final_df = pd.DataFrame()

for cmp in cmpList:
  cmp_df_list = []
  print('\nCompany: ', cmp)
  
  for item in items:
    url = urlPre + cmp + item
    dataframe_list = parse_url(url, cmp)
    
    if item is '':
      name = 'summary'
    else:
      name = item[1:len(item)]
   
    count = 1
    
    item_df = pd.DataFrame()
    
    for df in dataframe_list:
      if item == '/financials':
        df.set_index(df.columns[0], inplace=True)
        df = df.replace({pd.np.nan: None})
        df = df.unstack().to_frame().sort_index(level=1).T
        df.columns = df.columns.map("_".join)
        item_df = pd.concat([item_df, df], axis = 1)
     
   
    #print(item_df)
    #print(name, len(dataframe_list), 'tables ', item_df.shape[1], 'Columns')
    cmp_df_list.append(item_df)
    
#     filename = cmp + '_' + name + '.csv'
#     item_df.to_csv(filename, index = False)
#     !cp $filename drive/My\ Drive/Isenberg/
  
  
  
  cmp_df = pd.DataFrame()
  for df in cmp_df_list:
    cmp_df = pd.concat([cmp_df,df], axis = 1)
  
  # Causes problem when company name is "F" and it gets replaced everywhere in the column names. Better to do in parse_url
  # cmp_df.columns = cmp_df.columns.str.replace(cmp, "cmp")
  
  if columns == None:
    columns = cmp_df.columns.tolist()
 
  #cmp_df.sort_index(axis=1, inplace=True) # not required if string replace done at parsing time
  
  if (final_df.shape[0] != 0 and final_df.columns.equals(cmp_df.columns) == False ):
    # except when final_df is empty. This should return always True. Else column mistmatch (ignore this company)
    print('Column mismatch for company: ', cmp, ' Ignoring...')
    continue
    # MSFT: has current year (2020): typo
    # ADBE: has different dates
  else:
    towriteCmpList.append(cmp)
  
  #print('differences: ', cmp_df.columns.difference(final_df.columns).shape, final_df.columns.difference(cmp_df.columns).shape) #should always print 0
  
  final_df = pd.concat([final_df, cmp_df], ignore_index = True)

final_df['company'] = towriteCmpList  
final_df = final_df.set_index('company')
print('Final Data: ',final_df.shape)

filename = 'MarketWatchData.xlsx'
final_df.to_excel(filename)
!cp $filename drive/My\ Drive/Isenberg/

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).

Company:  TSLA

Company:  AAPL

Company:  NFLX

Company:  GM

Company:  F

Company:  FCAU

Company:  PCAR

Company:  MRK

Company:  PFE

Company:  JNJ

Company:  BMY

Company:  QCOM

Company:  MSFT
Column mismatch for company:  MSFT  Ignoring...

Company:  ADBE

Company:  INTC

Company:  GOOGL

Company:  CRM
Column mismatch for company:  CRM  Ignoring...

Company:  ORCL
Column mismatch for company:  ORCL  Ignoring...

Company:  IBM

Company:  FB

Company:  TWTR

Company:  AMZN

Company:  SCHW
Column mismatch for company:  SCHW  Ignoring...

Company:  ETFC
Column mismatch for company:  ETFC  Ignoring...

Company:  AMTD
Column mismatch for company:  AMTD  Ignoring...
Final Data:  (19, 342)


In [23]:
# Parsing data from yahoo (financials)

towriteCmpList = []

from google.colab import drive
drive.mount('drive')


items = ['/financials?p=', '/balance-sheet?p=', '/cash-flow?p=']
urlPre = 'https://finance.yahoo.com/quote/'

columns = None


final_df = pd.DataFrame()

for cmp in cmpList:
  cmp_df_list = []
  print('\nCompany: ', cmp)
  
  for item in items:
    url = urlPre + cmp + item + cmp
    dataframe_list = parse_url(url, cmp)
    
    
    name = item[1:len(item) - 3]
    
    count = 1
    
    item_df = getFinanceData(url)
     
   
    print(name, len(item_df), 'table ', item_df.shape[1], 'Columns')
    cmp_df_list.append(item_df)
    
#     filename = cmp + '_' + name + '.csv'
#     item_df.to_csv(filename, index = False)
#     !cp $filename drive/My\ Drive/Isenberg/
  
  
 
  cmp_df = pd.DataFrame()
  for df in cmp_df_list:
    cmp_df = pd.concat([cmp_df,df], axis = 1)
  
  # Causes problem when company name is "F" and it gets replaced everywhere in the column names. Better to do in parse_url
  # cmp_df.columns = cmp_df.columns.str.replace(cmp, "cmp")
  
  if columns == None:
    columns = cmp_df.columns.tolist()
 
  #cmp_df.sort_index(axis=1, inplace=True) # not required if string replace done at parsing time
  
  if (final_df.shape[0] != 0 and final_df.columns.equals(cmp_df.columns) == False ):
    # except when final_df is empty. This should return always True. Else column mistmatch (ignore this company)
    print('Column mismatch for company: ', cmp, ' Ignoring...')
    continue
    # MSFT: has current year (2020): typo
    # ADBE: has different dates
  else:
    towriteCmpList.append(cmp)
  
  #print('intersections: ', cmp_df.columns.difference(final_df.columns).shape, final_df.columns.difference(cmp_df.columns).shape) #should always print 0
  
  final_df = pd.concat([final_df, cmp_df], ignore_index = True)


final_df['company'] = towriteCmpList  
final_df = final_df.set_index('company')
print('Final Data: ',final_df.shape)

filename = 'YahooFinances.xlsx'
final_df.to_excel(filename)
!cp $filename drive/My\ Drive/Isenberg/

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).

Company:  TSLA
financials 1 tables  95 Columns
balance-sheet 1 tables  136 Columns
cash-flow 1 tables  145 Columns

Company:  AAPL
financials 1 tables  76 Columns
balance-sheet 1 tables  102 Columns
cash-flow 1 tables  116 Columns
Column mismatch for company:  AAPL  Ignoring...

Company:  NFLX
financials 1 tables  95 Columns
balance-sheet 1 tables  0 Columns
cash-flow 1 tables  0 Columns
Column mismatch for company:  NFLX  Ignoring...

Company:  GM
financials 1 tables  95 Columns
balance-sheet 1 tables  136 Columns
cash-flow 1 tables  145 Columns

Company:  F
financials 1 tables  95 Columns
balance-sheet 1 tables  136 Columns
cash-flow 1 tables  29 Columns
Column mismatch for company:  F  Ignoring...

Company:  FCAU
financials 1 tables  95 Columns
balance-sheet 1 tables  136 Columns
cash-flow 1 tables  145 Columns

Company:  PCAR
financials 1 tables  95 Columns
balance-sheet 

In [0]:
# Testing

towriteCmpList = []

from google.colab import drive
drive.mount('drive')


items = ['/financials?p=', '/balance-sheet?p=', '/cash-flow?p=']
urlPre = 'https://finance.yahoo.com/quote/'

columns = None


final_df = pd.DataFrame()

for cmp in cmpList:
  cmp_df_list = []
  print('\nCompany: ', cmp)
  
  for item in items:
    url = urlPre + cmp + item + cmp
    dataframe_list = parse_url(url, cmp)
    
    
    name = item[1:len(item) - 3]
    
    count = 1
    
    item_df = getFinanceData(url)
     
   
    print(name, len(item_df), 'table ', item_df.shape[1], 'Columns')
    cmp_df_list.append(item_df)
    
#     filename = cmp + '_' + name + '.csv'
#     item_df.to_csv(filename, index = False)
#     !cp $filename drive/My\ Drive/Isenberg/
  
  
 
  cmp_df = pd.DataFrame()
  for df in cmp_df_list:
    cmp_df = pd.concat([cmp_df,df], axis = 1)
  
  # Causes problem when company name is "F" and it gets replaced everywhere in the column names. Better to do in parse_url
  # cmp_df.columns = cmp_df.columns.str.replace(cmp, "cmp")
  
  if columns == None:
    columns = cmp_df.columns.tolist()
 
  #cmp_df.sort_index(axis=1, inplace=True) # not required if string replace done at parsing time
  
  if (final_df.shape[0] != 0 and final_df.columns.equals(cmp_df.columns) == False ):
    # except when final_df is empty. This should return always True. Else column mistmatch (ignore this company)
    print('Column mismatch for company: ', cmp, ' Ignoring...')
    print('differences: ', cmp_df.columns.difference(final_df.columns), final_df.columns.difference(cmp_df.columns)) 
    continue
    # MSFT: has current year (2020): typo
    # ADBE: has different dates

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).

Company:  TSLA
financials 1 table  95 Columns
balance-sheet 1 table  136 Columns


In [0]:
# response = requests.get(financeUrl, verify=False)
# soup = bs.BeautifulSoup(response.text)
# print(soup.prettify())